In [1]:
!nvidia-smi
!nvcc --version

Mon Mar 10 14:28:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -qU networkx adb-cloud-connector nx-arangodb 'langgraph==0.2.45' 'langchain-google-genai==2.0.4' langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into

In [3]:
# !pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com

In [ ]:
import os
from google.colab import userdata

# kaggle and google api setup
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_API_KEY")

In [5]:
!kaggle kernels output kjaybhaye/citation-data

Output file downloaded to /content/arxiv_metadata
Output file downloaded to /content/citations
Output file downloaded to /content/internal-references-v0.2.0-2019-03-01.json
Kernel log downloaded to /content/citation-data.log 


In [6]:
import numpy as np
import pandas as pd


## Get temporary cloud ArangoDb database and connect to it.

In [ ]:
from adb_cloud_connector import get_temp_credentials
from arango import ArangoClient
# from arango_datasets import Datasets

credentials = get_temp_credentials()

Log: requesting new credentials...


In [ ]:
db = ArangoClient(hosts=credentials["url"]).db(
    credentials["dbName"], credentials["username"], credentials["password"], verify=True
)

In [9]:
os.environ["DATABASE_HOST"] = credentials["url"]
os.environ["DATABASE_USERNAME"] = credentials["username"]
os.environ["DATABASE_PASSWORD"] = credentials["password"]
os.environ["DATABASE_NAME"] = credentials["dbName"]

## load dataset, create networkx graph and load it into ArangoDb database

In [10]:
meta_df = pd.read_csv("arxiv_metadata")
cit_df = pd.read_csv("citations")

In [11]:
meta_df.head()

,Unnamed: 0,id,authors,title,abstract,year
0,0,hep-ex/0001001,['TEXONO Collaboration'],A CsI(Tl) Scintillating Crystal Detector for t...,Scintillating crystal detector may offer som...,2012
1,1,hep-ex/0001002,['CLEO Collaboration'],Search for the Decay B0 -> D*0 gamma,We report results of a search for the rare r...,2012
2,2,hep-ex/0001003,"['S. Amato ', ' J. R. T. de Mello ', ' J. de M...",The E791 Parallel Architecture Data Acquisitio...,To collect data for the study of charm parti...,2008
3,3,hep-ex/0001004,"['FNAL E791 Collaboration: E. M. Aitala', ' K....",Observation of D-pi Production Correlations in...,We study the charge correlations between cha...,2008
4,4,hep-ex/0001005,"['A. Alavi-Harati', ' et al ']",Observation of the Decay $K_L\to \mu^+\mu^- \g...,We have observed the decay $K_L\to \mu^+\mu^...,2008


In [ ]:
meta_df.index = meta_df.id
meta_df = meta_df.drop("Unnamed: 0", axis=1)
# meta_df.year = meta_df.year.astype(int)
meta_df.year = meta_df.year.astype(object)

In [13]:
import networkx as nx
import nx_arangodb as nxadb

g = nx.DiGraph()

[14:30:19 +0000] [INFO]: NetworkX-cuGraph is available.
INFO:nx_arangodb:NetworkX-cuGraph is available.


In [ ]:
for k in cit_df["from"].unique():
    g.add_node(k, **dict(meta_df.loc[k]))

In [ ]:
for i in range(len(cit_df)):
    g.add_edge(cit_df["from"][i], cit_df["to"][i])

In [ ]:
# import matplotlib.pyplot as plt

# plot_options = {"node_size": 5, "with_labels": False, "width": 0.15}
# pos = nx.spring_layout(g, iterations=15, seed=1721)
# fig, ax = plt.subplots(figsize=(15, 9))
# nx.draw_networkx(g, pos=pos, ax=ax, **plot_options)

In [ ]:
G_adb = nxadb.DiGraph(incoming_graph_data=g, name="papers", overwrite_graph=True)

[14:34:37 +0000] [INFO]: Graph 'papers' created.
INFO:nx_arangodb:Graph 'papers' created.
[2025/03/10 14:34:38 +0000] [1027] [INFO] - adbnx_adapter: Instantiated ADBNX_Adapter with database 'TUTenjqxrkr09le770wvp309g'
INFO:adbnx_adapter:Instantiated ADBNX_Adapter with database 'TUTenjqxrkr09le770wvp309g'


Output()

Output()

[2025/03/10 14:34:59 +0000] [1027] [INFO] - adbnx_adapter: Created ArangoDB 'papers' Graph
INFO:adbnx_adapter:Created ArangoDB 'papers' Graph


## Create LLM agent

In [17]:
from typing import Annotated, Literal
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_core.messages.ai import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import langchain_google_genai as genai

from IPython.display import Image, display

from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain

import re

In [18]:
arango_graph = ArangoGraph(db)

In [19]:
global VERBOSE
VERBOSE = True

In [ ]:
@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """
    # print("tql tool")

    # llm = llm
    llm = genai.ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

    chain = ArangoGraphQAChain.from_llm(
        llm=llm, graph=arango_graph, verbose=True, allow_dangerous_requests=True
    )

    result = chain.invoke(query)

    return str(result["result"])

In [ ]:
@tool
def text_to_nx_algorithm_to_text(query: str):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """

    llm = genai.ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

    if VERBOSE:
        print("1) Generating NetworkX code")

    prompt = f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """

    text_to_nx = llm.invoke(prompt).content

    text_to_nx_cleaned = re.sub(
        r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE
    ).strip()

    if VERBOSE:
        print("-" * 10)
        print(text_to_nx_cleaned)
        print("-" * 10)

        print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        text_to_nx_final = text_to_nx
    except Exception as e:
        # print(f"EXEC ERROR: {e}")
        # return f"EXEC ERROR: {e}"

        # TODO: Consider experimenting with a code corrector!
        # attempt = 1
        # MAX_ATTEMPTS = 2
        messages = [HumanMessage(prompt), AIMessage(text_to_nx_final), HumanMessage(e)]
        # while attempt <= MAX_ATTEMPTS:
        # attempt += 1
        text_to_nx = llm.invoke(prompt).content
        text_to_nx_cleaned = re.sub(
            r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE
        ).strip()
        try:
            exec(text_to_nx_cleaned, global_vars, local_vars)
            text_to_nx_final = text_to_nx
        except Exception as e:
            messages.append(HumanMessage(text_to_nx_final))
            messages.append(AIMessage(text=str(e)))
            print(f"EXEC ERROR: {e}")
            return f"EXEC ERROR: {e}"

    FINAL_RESULT = local_vars["FINAL_RESULT"]

    if VERBOSE:
        print(f"FINAL_RESULT: {FINAL_RESULT}")
        print("-" * 10)

        print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.

        Your response:
    """).content

    return nx_to_text

In [22]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

tools = [text_to_aql_to_text, text_to_nx_algorithm_to_text]

llm = genai.ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

app = create_react_agent(llm, tools)

In [ ]:
def query_graph(query):
    inst = f"""You are an expert graph data analyst. You have access to a knowledge graph of academic papers represented in an ArangoDB database. You also have access to two powerful tools to help you answer user questions about this graph:

    1.  **text_to_aql_to_text:** This tool translates natural language questions into ArangoDB Query Language (AQL) queries, executes them against the database, and returns the results in natural language. Use this tool for queries that can be directly answered using AQL, such as retrieving paper information, counting papers, or performing simple graph traversals.
    2.  **text_to_nx_algorithm_to_text:** This tool allows you to execute NetworkX graph algorithms on the ArangoDB graph. Use this tool for complex graph analysis, such as calculating centrality measures, finding communities, or determining shortest paths. **Only use this tool when the question requires algorithms that cannot be directly handled by AQL.**

    **Your task is to analyze the user's question and use approapiate tools to answer the user's query.

    **Here are the schemas of the graph:**
    {arango_graph.schema}"""

    messages = [("system", inst), ("human", query)]

    final_state = app.invoke({"messages": messages})
    # final_state = app.invoke(HumanMessage(query))
    return final_state["messages"][-1].content

### Testing code

In [24]:
query_graph("Which is the most influential paper? Explain why")

1) Generating NetworkX code
----------
import networkx as nx

degree_centrality = nx.degree_centrality(G_adb)

most_influential_paper = max(degree_centrality, key=degree_centrality.get)

explanation = f"Paper with id '{most_influential_paper}' has the highest degree centrality ({degree_centrality[most_influential_paper]:.4f}), indicating it is directly connected to the most other papers in the network.  Therefore, it is the most influential based on direct connections."

FINAL_RESULT = (most_influential_paper, explanation)
----------

2) Executing NetworkX code


[14:39:43 +0000] [INFO]: Graph 'papers' load took 274.5475959777832s
INFO:nx_arangodb:Graph 'papers' load took 274.5475959777832s


FINAL_RESULT: ('papers_node/67643', "Paper with id 'papers_node/67643' has the highest degree centrality (0.0288), indicating it is directly connected to the most other papers in the network.  Therefore, it is the most influential based on direct connections.")
----------
3) Formulating final answer


"The paper with ID 'papers_node/67643' has the highest degree centrality (0.0288), meaning it's directly connected to the most other papers in the network and is therefore the most influential based on direct connections.\n"

In [25]:
global VERBOSE
VERBOSE = False

In [26]:
queries = [
    "What is the title of the paper with ID 'hep-ex/0001001'?",
    "Who are the authors of the paper with ID 'papers_node/0'?",
    "Which papers cite the paper 'papers_node/676'?",
    "Which papers does the paper 'papers_node/0' cite?",
    "Is there a citation path from 'papers_node/0' to 'papers_node/50'?",
    "What is the id of paper titled Physics Interplay of the LHC and the ILC'?",
    "How many times is 'papers_node/676' cited?",
    "Are 'papers_node/0' and 'papers_node/15' in the same citation network component?",
    "Which papers are most central in the citation network?",
    "Calculate the betweenness centrality of all papers.",
]

In [27]:
query_graph(queries[0])



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH papers_node
FOR p IN papers_node
  FILTER p.id == 'hep-ex/0001001'
  RETURN p.title

AQL Result:
['A CsI(Tl) Scintillating Crystal Detector for the Studies of Low Energy\n  Neutrino Interactions']

> Finished chain.


'The title of the paper with the ID \'hep-ex/0001001\' is "A CsI(Tl) Scintillating Crystal Detector for the Studies of Low Energy Neutrino Interactions".\n'

In [28]:
query_graph(queries[1])



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH papers_node
FOR p IN papers_node
  FILTER p._id == 'papers_node/0'
  RETURN p.authors

AQL Result:
["['Gorazd Cvetic ', ' Claudio O. Dib ', ' C.S. Kim  and J. D. Kim \\n  U. Federico Santa Maria', ' Chile; Yonsei U. and  Seoul Natl. U.', '\\n  Korea)']"]

> Finished chain.


"The authors of the paper with ID 'papers_node/0' are Gorazd Cvetic, Claudio O. Dib, C.S. Kim and J. D. Kim, U. Federico Santa Maria, Chile; Yonsei U. and Seoul Natl. U., Korea.\n"

In [29]:
query_graph(queries[2])



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH papers_node, papers_node_to_papers_node
FOR v, e IN INBOUND 'papers_node/676' papers_node_to_papers_node
    RETURN {title: v.title, authors: v.authors}

AQL Result:
[{'title': 'Cosmic ray signatures of massive relic particles', 'authors': "['Subir Sarkar ']"}, {'title': 'Particle Dark Matter: Evidence, Candidates and Constraints', 'authors': "['Gianfranco Bertone', ' Dan Hooper and Joseph Silk']"}]

> Finished chain.


"The papers that cite the paper with _id 'papers_node/676' are titled 'Cosmic ray signatures of massive relic particles' by Subir Sarkar and 'Particle Dark Matter: Evidence, Candidates and Constraints' by Gianfranco Bertone, Dan Hooper and Joseph Silk.\n"

In [30]:
query_graph(queries[3])



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH papers_node, papers_node_to_papers_node
FOR v, e IN OUTBOUND 'papers_node/0' papers_node_to_papers_node
RETURN v.title

AQL Result:
['On Lepton Flavor Violation in Tau Decays', 'Three-flavor MSW solutions of the solar neutrino problem', None, None, None, 'Status of the MSW Solutions to the Solar Neutrino Problem', 'Muonium-Antimuonium oscillations and massive Majorana neutrinos', 'Where do we stand with solar neutrino oscillations?', None, None]

> Finished chain.


"The paper 'papers_node/0' cites the following papers: 'On Lepton Flavor Violation in Tau Decays', 'Three-flavor MSW solutions of the solar neutrino problem', 'Status of the MSW Solutions to the Solar Neutrino Problem', 'Muonium-Antimuonium oscillations and massive Majorana neutrinos', and 'Where do we stand with solar neutrino oscillations?'. Some cited papers do not have titles.\n"

In [33]:
query_graph(queries[4])

"Yes, there is a citation path from 'papers_node/0' to 'papers_node/50'.\n"

In [34]:
query_graph(queries[5])



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH papers_node
FOR p IN papers_node
  FILTER p.title == "Physics Interplay of the LHC and the ILC"
  RETURN p.id

AQL Result:
['hep-ph/0410364']

> Finished chain.


'The id of the paper titled "Physics Interplay of the LHC and the ILC" is hep-ph/0410364.\n'

In [35]:
query_graph(queries[6])



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH papers_node_to_papers_node
RETURN COUNT(FOR edge IN papers_node_to_papers_node FILTER edge._to == 'papers_node/676' RETURN edge)

AQL Result:
[2]

> Finished chain.


"The paper 'papers_node/676' is cited 2 times.\n"

In [36]:
query_graph(queries[7])



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH papers_node, papers_node_to_papers_node
FOR v, e, p IN OUTBOUND SHORTEST_PATH 'papers_node/0' TO 'papers_node/15' papers_node_to_papers_node
RETURN p

AQL Query Execution Error: 
AQL: SHORTEST_PATH only has one or two return variables near 'FOR v, e, p IN OUTBOUND SHORTEST...' at position 3:1 (while parsing)

AQL Query (2):
WITH papers_node, papers_node_to_papers_node
FOR v, e IN OUTBOUND SHORTEST_PATH 'papers_node/0' TO 'papers_node/15' papers_node_to_papers_node
RETURN {vertex:v, edge:e}

AQL Result:
[{'vertex': {'_key': '0', '_id': 'papers_node/0', '_rev': '_jW_4vvW---', 'id': 'hep-ph/0504126', 'authors': "['Gorazd Cvetic ', ' Claudio O. Dib ', ' C.S. Kim  and J. D. Kim \\n  U. Federico Santa Maria', ' Chile; Yonsei U. and  Seoul Natl. U.', '\\n  Korea)']", 'title': 'Muonium-antimuonium conversion in models with heavy neutrinos', 'abstract': '  We study muonium-antimuonium conversion and mu+ e- to mu- e+ scattering\nwi

"Since there is no path between 'papers_node/0' and 'papers_node/15', they are not in the same citation network component. Note that the previous response indicated that 'papers_node/15' was not in the dataset, which is likely the reason why no path could be found.\n"

## create gradio interface

In [37]:
!pip install -qU gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.6 MB/s eta 0:00:00


In [38]:
import gradio as gr

gr.Interface(fn=query_graph, inputs="text", outputs="text").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df48ae499b72c36c92.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
